In [1]:
#setup data
import pandas as pd
import numpy as np
import math
import random
from sklearn import preprocessing

ignore_features = ['25', '26', '8', '58', '20', '23', '29', '31', '32', '35']
toencode = {}
f = open("field_types.txt", 'r')
for line in f.readlines():
    line = line.strip().split(" ")
    if line[1] != "numeric" and line[0] not in ignore_features:
        toencode[line[0]] = len(line) - 1
        

f1 = open("data.csv")
data_df = pd.read_csv(f1, sep=',', header = 'infer', low_memory=False)
data_df = data_df.drop(ignore_features, axis=1)
label_df = data_df[['label']]
data_df = data_df.drop('label', axis=1)
label = np.array(label_df)


f2 = open("quiz.csv")
quiz_df = pd.read_csv(f2, sep=',', header = 'infer', low_memory=False)
quiz_df = quiz_df.drop(ignore_features, axis=1)


In [2]:
#getting test and train
data_end = np.shape(data_df)[0]
df = data_df.append(quiz_df)
df_trans = pd.get_dummies(df, columns=toencode.keys(), dummy_na=False)
data = np.array(df_trans)

data, quiz = data[:data_end], data[data_end:]

In [3]:
#splitting to features and labels by training and test
def splitdata(data, label):
    n = data.shape[0]
    train_num = int(math.ceil(n*.8))
    indices = random.sample(xrange(n), n)

    training_idx, test_idx = indices[train_num:], indices[:train_num]
    training, test = data[training_idx,:], data[test_idx,:]
    lab, testlabel = label[training_idx,:], label[test_idx,:]
    
    return training, lab, test, testlabel

train, lab, test, testlabel = splitdata(data, label)

In [4]:
#evaluator
def eval(preds, testlabels):
    if len(preds) == len(testlabels):
        error = np.count_nonzero(preds != testlabels) / np.float(len(preds))
        return 1 - error

In [5]:
#unraveling
testlabelnum = np.ravel(testlabel)
labnum = np.ravel(lab)

In [8]:
# scalar weighting
scaler = preprocessing.StandardScaler().fit(train)
train = scaler.transform(train) 
test = scaler.transform(test) 
quiz = scaler.transform(quiz) 

In [9]:
#logistic regression (0.87656)
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

ylogregtest = logreg.fit(train, lab).predict(test)

print eval(ylogregtest, testlabelnum)

/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
eval(ylogregtesttrans,testlabelnum)

0.876564501823


In [11]:
#adaboost on dec tree max_dept = 10 (0.901)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
adaboost = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10),n_estimators=600, learning_rate=1)

yadaboosttest = adaboost.fit(train, lab).predict(test)
print eval(yadaboosttest, testlabelnum)

0.916921257515


/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#adaboost on dec tree max_dept = 20' learning_rate = 2 (0.9197)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
adaboost20lr = AdaBoostClassifier(DecisionTreeClassifier(max_depth=30),n_estimators=1000, learning_rate=.75)

yadaboosttest20lr = adaboost20lr.fit(train, lab).predict(test)
print eval(yadaboosttest20lr, testlabelnum)

In [ ]:
import csv
csvfile = "adaboost.csv"

with open(csvfile,"w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in yadaboostquiz:
        writer.writerow([val])